In [29]:
# coding: utf-8
import requests
import json
import datetime
import pandas as pd
from IPython.display import display
from pandas.io.json import json_normalize


url = 'https://api.overwatchleague.com/matches'

print('Fetching data from {}...'.format(url))
r = requests.get(url)
match_data = r.json()

print('Converting json to DataFrame...')
content = match_data['content']
n_matches = len(content)
match_ids = [content[i]['id'] for i in range(n_matches)]

Fetching data from https://api.overwatchleague.com/matches...
Converting json to DataFrame...


In [429]:
teamsDict ={}

url = "https://api.overwatchleague.com/teams"
# print('Fetching data from {}...'.format(url))
r = requests.get(url)
match_data = r.json()
norm = json_normalize(match_data, ["competitors"])
for tm in range(len(norm)):
#     print(str(norm["competitor"][tm]["id"])+ ": "+norm["competitor"][tm]["abbreviatedName"])
    team={norm["competitor"][tm]["id"]: norm["competitor"][tm]["abbreviatedName"]}
    teamsDict.update(team)
# print(teamsDict)





playersDict={}

for key, value in teamsDict.items(): 
    url = 'https://api.overwatchleague.com/teams/'+str(key)
#     print('Fetching data from {}...'.format(url))
    r = requests.get(url)
    match_data = r.json()
    norm = json_normalize(match_data, ["players"])
    for ply in range(len(norm["id"])):
#         print(str(norm["id"][ply])+ ": "+norm["name"][ply])
        player={norm["id"][ply]: norm["name"][ply]}
        playersDict.update(player)
# print(playersDict)



mapsDict={}
url = "https://api.overwatchleague.com/maps"
r = requests.get(url)
match_data = r.json()
norm = json_normalize(match_data)
for mp in range(len(norm)):
    thismap={norm["guid"][mp]: norm["id"][mp]}
    mapsDict.update(thismap)
# print(mapsDict)

{'0x0800000000000B56': 'Havana', '0x080000000000005B': 'temple-of-anubis', '0x08000000000000D4': 'kings-row', '0x0800000000000165': 'hanamura', '0x0800000000000184': 'gibraltar', '0x08000000000001D4': 'numbani', '0x08000000000001DB': 'volskaya', '0x08000000000002AF': 'hollywood', '0x08000000000002C3': 'dorado', '0x08000000000004B7': 'nepal', '0x08000000000005BB': 'route-66', '0x0800000000000661': '', '0x0800000000000662': 'lijiang', '0x080000000000066D': 'ilios', '0x0800000000000688': '', '0x080000000000068D': 'eichenwalde', '0x080000000000069E': 'oasis', '0x08000000000006AB': 'hollywood', '0x08000000000006B1': 'kings-row', '0x08000000000006B3': 'estadio-das-ras', '0x08000000000006B5': 'hanamura', '0x08000000000006B7': 'lijiang', '0x08000000000006C7': '', '0x08000000000006C9': '', '0x08000000000006D1': 'ecopoint-antarctica', '0x08000000000006D3': 'horizon-lunar-colony', '0x08000000000006FE': 'kings-row', '0x0800000000000705': 'necropolis', '0x080000000000070C': 'black-forest', '0x08000

In [437]:
# print(len(content[0]["games"]))

# for map_num in range(1, len(content[0]["games"])+1):
#     print(map_num)

print(len(content[0]["games"]))

url = 'https://api.overwatchleague.com/stats/matches/'+str(match_ids[0])+"/maps/1"
print('Fetching data from {}...'.format(url))
r = requests.get(url)
match_data = r.json()

df = pd.DataFrame.from_dict(json_normalize(match_data), orient='columns')
df.drop(columns=["stats", "teams"])

for val in df["map_id"]:
    df= df.replace(val, mapsDict[val])
    print(val, mapsDict[val])

#adding the match stats category
norm = json_normalize(match_data, ["stats"])
df.insert(len(df.columns), norm["name"][0], norm["value"])

#adding the team stats categories
norm = json_normalize(match_data, ["teams"])

team1=teamsDict[norm["esports_team_id"][0]]
df.insert(len(df.columns), "team_1", team1)
df.insert(len(df.columns), team1+"_deaths", norm["stats"][0][0]["value"])

team2=teamsDict[norm["esports_team_id"][1]]
df.insert(len(df.columns), "team_2", team2)
df.insert(len(df.columns), team2+"_deaths", norm["stats"][1][0]["value"])

for player in range(len(norm["players"][0])):
    thisPlayer=playersDict[norm["players"][0][player]["esports_player_id"]]
    for data in range(len(norm["players"][0][player]["stats"])):
        print(team1+"_"+thisPlayer+"_"+str(norm["players"][0][player]["stats"][data]["name"])+":", norm["players"][0][player]["stats"][data]["value"])
#         df.insert(len(df.columns), team1+"_"+thisPlayer+"_"+str(norm["players"][0][player]["stats"][data]["name"]), norm["players"][0][player]["stats"][data]["value"])

# df.describe()
# for pl in 
print("--------")

player=0
print(norm["players"][0][0])
print("--------")


    
print(df)

4
Fetching data from https://api.overwatchleague.com/stats/matches/20301/maps/1...
0x080000000000066D ilios
SEO_tobi_damage: 5822.933597564697
SEO_tobi_deaths: 3
SEO_tobi_eliminations: 13
SEO_tobi_healing: 4155.94883453846
SEO_ZUNBA_damage: 12419.718645602465
SEO_ZUNBA_deaths: 1
SEO_ZUNBA_eliminations: 23
SEO_ryujehong_damage: 10959.600513458252
SEO_ryujehong_deaths: 1
SEO_ryujehong_eliminations: 18
SEO_ryujehong_healing: 8384.268975377083
SEO_FITS_damage: 19436.45032286644
SEO_FITS_eliminations: 26
SEO_Fleta_damage: 7405.2210302352905
SEO_Fleta_deaths: 1
SEO_Fleta_eliminations: 25
SEO_Fleta_healing: 8407.800655603409
SEO_Marve1_damage: 7453.8512561917305
SEO_Marve1_deaths: 4
SEO_Marve1_eliminations: 16
--------
{'esports_player_id': 4104, 'stats': [{'name': 'damage', 'value': 5822.933597564697, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 3, 'id': '0x086000000000002A'}, {'name': 'eliminations', 'value': 13, 'id': '0x086000000000001F'}, {'name': 'healing', 'value': 4155.948

In [ ]:
[{'esports_team_id': 4409, 
  'stats': 
      [{'name': 'deaths', 
        'value': 10, 
        'id': '0x086000000000002A'}], 
  'players': 
      [{'esports_player_id': 4104, 
        'stats': 
            [{'name': 'damage', 
              'value': 5822.933597564697, 
              'id': '0x08600000000001BF'}, 
             {'name': 'deaths', 
              'value': 3, 
              'id': '0x086000000000002A'}, 
             {'name': 'eliminations', 
              'value': 13, 
              'id': '0x086000000000001F'}, 
             {'name': 'healing', 
              'value': 4155.94883453846, 
              'id': '0x08600000000001C1'}], 
        'heroes': 
            [{'hero_id': '0x02E0000000000079', 
              'name': 'lucio', 
              'stats': 
                  [{'name': 'damage', 
                    'value': 5822.933597564697, 
                    'id': '0x08600000000001BF'}, 
                   {'name': 'deaths', 
                    'value': 3, 
                    'id': '0x086000000000002A'}, 
                   {'name': 'eliminations', 
                    'value': 13, 
                    'id': '0x086000000000001F'}, 
                   {'name': 'healing', 
                    'value': 4155.94883453846, 
                    'id': '0x08600000000001C1'}]}]}, 
       {'esports_player_id': 4098, 'stats': [{'name': 'damage', 'value': 12419.718645602465, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 1, 'id': '0x086000000000002A'}, {'name': 'eliminations', 'value': 23, 'id': '0x086000000000001F'}], 'heroes': [{'hero_id': '0x02E000000000007A', 'name': 'dva', 'stats': [{'name': 'damage', 'value': 12419.718645602465, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 1, 'id': '0x086000000000002A'}, {'name': 'eliminations', 'value': 23, 'id': '0x086000000000001F'}]}]}, {'esports_player_id': 4105, 'stats': [{'name': 'damage', 'value': 10959.600513458252, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 1, 'id': '0x086000000000002A'}, {'name': 'eliminations', 'value': 18, 'id': '0x086000000000001F'}, {'name': 'healing', 'value': 8384.268975377083, 'id': '0x08600000000001C1'}], 'heroes': [{'hero_id': '0x02E0000000000020', 'name': 'zenyatta', 'stats': [{'name': 'damage', 'value': 10959.600513458252, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 1, 'id': '0x086000000000002A'}, {'name': 'eliminations', 'value': 18, 'id': '0x086000000000001F'}, {'name': 'healing', 'value': 8384.268975377083, 'id': '0x08600000000001C1'}]}]}, {'esports_player_id': 8831, 'stats': [{'name': 'damage', 'value': 19436.45032286644, 'id': '0x08600000000001BF'}, {'name': 'eliminations', 'value': 26, 'id': '0x086000000000001F'}], 'heroes': [{'hero_id': '0x02E0000000000068', 'name': 'zarya', 'stats': [{'name': 'damage', 'value': 19436.45032286644, 'id': '0x08600000000001BF'}, {'name': 'eliminations', 'value': 26, 'id': '0x086000000000001F'}]}]}, {'esports_player_id': 4631, 'stats': [{'name': 'damage', 'value': 7405.2210302352905, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 1, 'id': '0x086000000000002A'}, {'name': 'eliminations', 'value': 25, 'id': '0x086000000000001F'}, {'name': 'healing', 'value': 8407.800655603409, 'id': '0x08600000000001C1'}], 'heroes': [{'hero_id': '0x02E0000000000195', 'name': 'brigitte', 'stats': [{'name': 'damage', 'value': 7405.2210302352905, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 1, 'id': '0x086000000000002A'}, {'name': 'eliminations', 'value': 25, 'id': '0x086000000000001F'}, {'name': 'healing', 'value': 8407.800655603409, 'id': '0x08600000000001C1'}]}]}, {'esports_player_id': 8218, 'stats': [{'name': 'damage', 'value': 7453.8512561917305, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 4, 'id': '0x086000000000002A'}, {'name': 'eliminations', 'value': 16, 'id': '0x086000000000001F'}], 'heroes': [{'hero_id': '0x02E0000000000007', 'name': 'reinhardt', 'stats': [{'name': 'damage', 'value': 3750.980827331543, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 2, 'id': '0x086000000000002A'}, {'name': 'eliminations', 'value': 6, 'id': '0x086000000000001F'}]}, {'hero_id': '0x02E0000000000009', 'name': 'winston', 'stats': [{'name': 'damage', 'value': 3702.8704288601875, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 2, 'id': '0x086000000000002A'}, {'name': 'eliminations', 'value': 10, 'id': '0x086000000000001F'}]}]}]}, {'esports_team_id': 4403, 'stats': [{'name': 'deaths', 'value': 31, 'id': '0x086000000000002A'}], 'players': [{'esports_player_id': 8236, 'stats': [{'name': 'damage', 'value': 13675.524320542812, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 6, 'id': '0x086000000000002A'}, {'name': 'eliminations', 'value': 8, 'id': '0x086000000000001F'}], 'heroes': [{'hero_id': '0x02E0000000000068', 'name': 'zarya', 'stats': [{'name': 'damage', 'value': 13675.524320542812, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 6, 'id': '0x086000000000002A'}, {'name': 'eliminations', 'value': 8, 'id': '0x086000000000001F'}]}]}, {'esports_player_id': 4654, 'stats': [{'name': 'damage', 'value': 5163.786023616791, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 6, 'id': '0x086000000000002A'}, {'name': 'eliminations', 'value': 6, 'id': '0x086000000000001F'}, {'name': 'healing', 'value': 7026.607188820839, 'id': '0x08600000000001C1'}], 'heroes': [{'hero_id': '0x02E0000000000195', 'name': 'brigitte', 'stats': [{'name': 'damage', 'value': 4801.896150112152, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 5, 'id': '0x086000000000002A'}, {'name': 'eliminations', 'value': 6, 'id': '0x086000000000001F'}, {'name': 'healing', 'value': 7026.607188820839, 'id': '0x08600000000001C1'}]}, {'hero_id': '0x02E00000000001CA', 'name': 'wreckingball', 'stats': [{'name': 'damage', 'value': 361.8898735046387, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 1, 'id': '0x086000000000002A'}]}]}, {'esports_player_id': 4093, 'stats': [{'name': 'damage', 'value': 7129.036652028561, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 7, 'id': '0x086000000000002A'}, {'name': 'eliminations', 'value': 5, 'id': '0x086000000000001F'}], 'heroes': [{'hero_id': '0x02E000000000013E', 'name': 'orisa', 'stats': [{'name': 'damage', 'value': 471, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 1, 'id': '0x086000000000002A'}]}, {'hero_id': '0x02E0000000000007', 'name': 'reinhardt', 'stats': [{'name': 'damage', 'value': 3893.5265789031982, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 4, 'id': '0x086000000000002A'}, {'name': 'eliminations', 'value': 2, 'id': '0x086000000000001F'}]}, {'hero_id': '0x02E0000000000009', 'name': 'winston', 'stats': [{'name': 'damage', 'value': 2764.5100731253624, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 2, 'id': '0x086000000000002A'}, {'name': 'eliminations', 'value': 3, 'id': '0x086000000000001F'}]}]}, {'esports_player_id': 4653, 'stats': [{'name': 'damage', 'value': 10115.769534111023, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 4, 'id': '0x086000000000002A'}, {'name': 'eliminations', 'value': 8, 'id': '0x086000000000001F'}, {'name': 'healing', 'value': 7574.7989110946655, 'id': '0x08600000000001C1'}], 'heroes': [{'hero_id': '0x02E0000000000020', 'name': 'zenyatta', 'stats': [{'name': 'damage', 'value': 10115.769534111023, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 4, 'id': '0x086000000000002A'}, {'name': 'eliminations', 'value': 8, 'id': '0x086000000000001F'}, {'name': 'healing', 'value': 7574.7989110946655, 'id': '0x08600000000001C1'}]}]}, {'esports_player_id': 4649, 'stats': [{'name': 'damage', 'value': 9802.382920056581, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 5, 'id': '0x086000000000002A'}, {'name': 'eliminations', 'value': 10, 'id': '0x086000000000001F'}], 'heroes': [{'hero_id': '0x02E000000000007A', 'name': 'dva', 'stats': [{'name': 'damage', 'value': 9802.382920056581, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 5, 'id': '0x086000000000002A'}, {'name': 'eliminations', 'value': 10, 'id': '0x086000000000001F'}]}]}, {'esports_player_id': 4841, 'stats': [{'name': 'damage', 'value': 7101.201858520508, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 3, 'id': '0x086000000000002A'}, {'name': 'eliminations', 'value': 6, 'id': '0x086000000000001F'}, {'name': 'healing', 'value': 5643.376592993736, 'id': '0x08600000000001C1'}], 'heroes': [{'hero_id': '0x02E0000000000079', 'name': 'lucio', 'stats': [{'name': 'damage', 'value': 7101.201858520508, 'id': '0x08600000000001BF'}, {'name': 'deaths', 'value': 3, 'id': '0x086000000000002A'}, {'name': 'eliminations', 'value': 6, 'id': '0x086000000000001F'}, {'name': 'healing', 'value': 5643.376592993736, 'id': '0x08600000000001C1'}]}]}]}]

In [441]:
myData = []

for i, m_id in enumerate(match_ids):
    for map_num in range(1, len(content[i]["games"])+1):
        url = 'https://api.overwatchleague.com/stats/matches/'+str(m_id)+"/maps/"+str(map_num)
        print('Fetching data from {}...'.format(url))
        r = requests.get(url)
        reads=False
        try:
            r.json()
            reads=True
        except ValueError as e:
            print('invalid json: %s' % e)
            reads=False
        if reads is True:
            match_data = r.json()
            df = pd.DataFrame.from_dict(json_normalize(match_data), orient='columns')
            df=df.drop(columns=["stats", "teams"])
            
            for val in df["map_id"]:
                df=df.replace(val, mapsDict[val])
                
            norm = json_normalize(match_data, ["stats"])
            df.insert(len(df.columns), norm["name"][0], norm["value"])
            #adding the team stats categories
            norm = json_normalize(match_data, ["teams"])

            team1=teamsDict[norm["esports_team_id"][0]]
            df.insert(len(df.columns), "team_1", team1)
            df.insert(len(df.columns), team1+"_deaths", norm["stats"][0][0]["value"])

            team2=teamsDict[norm["esports_team_id"][1]]
            df.insert(len(df.columns), "team_2", team2)
            df.insert(len(df.columns), team2+"_deaths", norm["stats"][1][0]["value"])
            myData.append(df)

print("--------------DONE!----------------")

Fetching data from https://api.overwatchleague.com/stats/matches/20301/maps/1...
Fetching data from https://api.overwatchleague.com/stats/matches/20301/maps/2...
Fetching data from https://api.overwatchleague.com/stats/matches/20301/maps/3...
Fetching data from https://api.overwatchleague.com/stats/matches/20301/maps/4...
Fetching data from https://api.overwatchleague.com/stats/matches/20302/maps/1...
Fetching data from https://api.overwatchleague.com/stats/matches/20302/maps/2...
Fetching data from https://api.overwatchleague.com/stats/matches/20302/maps/3...
Fetching data from https://api.overwatchleague.com/stats/matches/20303/maps/1...
Fetching data from https://api.overwatchleague.com/stats/matches/20303/maps/2...
Fetching data from https://api.overwatchleague.com/stats/matches/20303/maps/3...
Fetching data from https://api.overwatchleague.com/stats/matches/20303/maps/4...
Fetching data from https://api.overwatchleague.com/stats/matches/20304/maps/1...
Fetching data from https://a

IndexError: list index out of range

In [442]:
result=pd.concat(myData)            
result.to_csv('OWL_matches.csv', index=False, sep=";")

# import json
# with open('data.json', 'w') as outfile:
#     json.dump(match_data, outfile)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [65]:
result["teams"]

0    [{'esports_team_id': 4409, 'stats': [{'name': ...
0    [{'esports_team_id': 4409, 'stats': [{'name': ...
0    [{'esports_team_id': 4409, 'stats': [{'name': ...
0    [{'esports_team_id': 4409, 'stats': [{'name': ...
0    [{'esports_team_id': 4402, 'stats': [{'name': ...
0    [{'esports_team_id': 4402, 'stats': [{'name': ...
0    [{'esports_team_id': 4402, 'stats': [{'name': ...
0    [{'esports_team_id': 7698, 'stats': [{'name': ...
0    [{'esports_team_id': 7698, 'stats': [{'name': ...
0    [{'esports_team_id': 7698, 'stats': [{'name': ...
0    [{'esports_team_id': 7698, 'stats': [{'name': ...
0    [{'esports_team_id': 7695, 'stats': [{'name': ...
0    [{'esports_team_id': 7695, 'stats': [{'name': ...
0    [{'esports_team_id': 7695, 'stats': [{'name': ...
0    [{'esports_team_id': 4409, 'stats': [{'name': ...
0    [{'esports_team_id': 4409, 'stats': [{'name': ...
0    [{'esports_team_id': 4409, 'stats': [{'name': ...
0    [{'esports_team_id': 4409, 'stats': [{'name': ...
0    [{'es